In [8]:
import pandas as pd

In [9]:
usd = {'eth':{}}

df = pd.read_csv('input/yahoo finance/ETH-USD.csv')

for row in df.iterrows():
    x = row[1]
    usd['eth'][x['Date']] = x['Open']

In [10]:
df = pd.read_excel('input/transactions.xlsx',header=2,keep_default_na=False)

df['price'] = df.apply(lambda x: usd[x['symbol']][x['timestamp'].strftime('%Y-%m-%d')],axis=1)

df['proceeds'] = df.apply(lambda x: x['price'] * -x['qty'] if x['qty'] < 0 else 0,axis=1)

df = df.sort_values(by='timestamp',ascending=False)
df = df.reset_index(drop=True)

In [11]:
def find_basis(df,irow, debug=False):

    target_row = df.iloc[irow]

    if target_row['qty'] >= 0:
        raise Exception('Can only calculate basis from debits')

    to_base = target_row['qty'] * -1
    basis_src = 0
    basis = 0
    
    if debug:
        print(f"bases for #{irow} ({target_row['qty']}):")
        print('---------------')
        print('row: qty (bal)')
        print('---------------')

    while to_base > 0:
        irow += 1
        if irow >= len(df):
            raise Exception('Insufficient basis')

        row = df.iloc[irow]

        basis_src += row['qty']

        if debug:
            print(f"#{irow}: {row['qty']} ({basis_src})")
        
        if basis_src > 0:
            to_take = basis_src
            if to_base < basis_src:
                to_take = to_base

            if debug:
                print(f"taking {to_take} @ {row['price']}")

            to_base -= to_take
            basis_src -= to_take

            basis += to_take * row['price']

    return basis

# example

find_basis(df,35,True)

bases for #35 (-1.56402337):
---------------
row: qty (bal)
---------------
#36: 1.56402337 (1.56402337)
taking 1.56402337 @ 2467.188477


3858.7404362227076

In [12]:
df['basis'] = df.apply(lambda x: find_basis(df,x.name) if x['qty'] < 0 else 0,axis=1)
df['gains'] = df['proceeds'] - df['basis']

df

,timestamp,source,symbol,qty,special type,price,proceeds,basis,gains
0,2022-05-21 18:14:58,coinbase wallet-1,eth,-0.003276,,1961.017944,6.424963,6.424963,0.000000
1,2022-05-21 18:14:58,coinbase wallet-1-weth,eth,-0.018750,,1961.017944,36.769086,36.769086,0.000000
2,2022-05-21 18:14:58,coinbase wallet-1-weth,eth,0.150000,,1961.017944,0.000000,0.000000,0.000000
3,2022-05-19 21:12:04,coinbase wallet-1,eth,-0.000802,,1916.149536,1.536454,1.941784,-0.405330
4,2022-05-19 21:09:53,coinbase wallet-1,eth,-0.001163,,1916.149536,2.228689,2.816637,-0.587948
5,2022-05-19 21:09:41,coinbase wallet-1,eth,-0.010265,,1916.149536,19.670110,24.859262,-5.189152
6,2022-05-17 20:59:58,coinbase wallet-1,eth,-0.006936,,2022.882324,14.031306,16.797254,-2.765949
7,2022-05-14 17:09:44,coinbase wallet-1,eth,-0.003000,,2014.280640,6.042842,7.264941,-1.222099
8,2022-05-14 17:09:44,coinbase wallet-1,eth,-0.000482,,2014.280640,0.970608,1.166903,-0.196295
9,2022-05-14 17:02:12,coinbase wallet-1,eth,-0.001699,,2014.280640,3.422182,4.114281,-0.692099


In [13]:
output = df.copy()

output = output.rename(columns={'gains': f"gains ({df['gains'].sum():.2f})"})

outfile = 'output/report.csv'
output.to_csv(outfile,index=False)
outfile

'output/report.csv'